## File I/O

### Filesystem

In [ ]:
# Print working directory
pwd()

In [ ]:
# Get files and directories in current directory
readdir()

In [ ]:
# Get home directory
ENV["HOME"]

In [ ]:
# Another way of getting the home directory
homedir()

In [ ]:
# Create a string holding the full path to ~/tmp/julia
JULIATMPDIR = joinpath(homedir(), "tmp/julia")

In [ ]:
# If ~/tmp/julia directory doesn't exist, then create it
if !isdir(JULIATMPDIR)
  mkpath(JULIATMPDIR)
end

In [ ]:
# Create the directory ~/tmp/julia/dir01/dir02
mkpath(joinpath(JULIATMPDIR, "dir01/dir02"))

In [ ]:
# Delete the directory ~/tmp/julia/dir01/dir02
rm(joinpath(JULIATMPDIR, "dir01"), recursive=true)

### Data I/O

####  write (and read) a vector to (and from) a file

In [ ]:
# Define a vector
x = [2.5, 3.0, 1.0]

# Create a string holding the full path to ~/tmp/julia
JULIATMPDIR = joinpath(homedir(), "tmp/julia")

# If ~/tmp/julia directory doesn't exist, then create it
if !isdir(JULIATMPDIR)
  mkpath(JULIATMPDIR)
end

# Create a string holding the filename on which to operate
DATAFILE = joinpath(JULIATMPDIR, "out01.txt")

# Write x to one-column DATAFILE
writedlm(DATAFILE, x)

# Read one-column DATAFILE into vector y
y = readdlm(DATAFILE, ' ')

#### write (and read) a vector to (and from) a file line by line

In [ ]:
# Define a vector
x = [2.5, 3.0, 1.0]

# Create a string holding the full path to ~/tmp/julia
JULIATMPDIR = joinpath(homedir(), "tmp/julia")

# If ~/tmp/julia directory doesn't exist, then create it
if !isdir(JULIATMPDIR)
  mkpath(JULIATMPDIR)
end

# Create a string holding the filename on which to operate
DATAFILE = joinpath(JULIATMPDIR, "out02.txt")

# Write x to one-column DATAFILE line by line
outstream = open(DATAFILE, "w+")

for i = 1:length(x)
  write(outstream, string(x[i], "\n"))
end

close(outstream)

# Read one-column DATAFILE line by line into vector y
instream = open(DATAFILE, "r")

y = Array(Float64, countlines(DATAFILE))
i = 0

while !eof(instream)
    y[i+=1] = parse(Float64, readline(instream))
end

close(instream)

y

### JLD - To save julia objects to file and retrieve back

If you just want to save a few variables and don't care to use the more advanced features, then a simple syntax is:

In [9]:
using JLD
t = 15
z = [1,3]
save("/tmp/myfile.jld", "t", t, "arr", z)

Here we're explicitly saving t and z as "t" and "arr" within myfile.jld.

To load back the variables, 

In [10]:
d = load("/tmp/myfile.jld")

Dict{String,Any} with 2 entries:
  "arr" => [1,3]
  "t"   => 15

In [11]:
d["arr"]

2-element Array{Int64,1}:
 1
 3

Or you can be more specific and just request particular variables of interest. 

In [15]:
z = load("/tmp/myfile.jld", "arr")

2-element Array{Int64,1}:
 1
 3

In [ ]:
There are some convenience macros like `@save` and `@load` 

In [17]:
@save "/tmp/myfile1.jld" t z

In [18]:
@save "/tmp/myfile2.jld" # will save all the variables in global scope

In [19]:
@load "/tmp/myfile2.jld"

3-element Array{Symbol,1}:
 :d
 :t
 :z

The limitation of the above macros is that they cannot be used inside functions, since the functions are compiled to machine code before evaluation, no new variables can be introduced during runtime.

More fine-grained control can be obtained using functional syntax:

In [20]:
jldopen("mydata.jld", "w") do file
    write(file, "A", A)  # alternatively, say "@write file A"
end

c = jldopen("mydata.jld", "r") do file
    read(file, "A")
end

LoadError: UndefVarError: A not defined